In [1]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [2]:
# Add simple group
from src.data_processors_lib.simple_processors import SimpleGroup

cart_events = EventsNode(
    SimpleGroup({
        "event_name": "add_to_cart",
        "filter": lambda df, schema: df[schema.event_name].isin(["cart"]),
    })
)
graph.add_node(
    node=cart_events,
    parents=[graph.root]
)
# result = graph.combine(cart_events)
# result.to_dataframe().head()

In [4]:
from src.data_processors_lib.simple_processors import StartEndEvents

start_end_events = EventsNode(
    StartEndEvents(
        {
            'user_col': 'user_id',
            'event_col': 'event_name',
            'time_col': 'event_timestamp',
            'type_col': 'event_type'
        })
)

graph.add_node(
    node=start_end_events,
    parents=[cart_events]
)

# from src.graph.p_graph import MergeNode
# merge = MergeNode()


result = graph.combine(
    node=start_end_events
)
result.to_dataframe()

/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.__events = result_left_part.append(
/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.__events = result_left_part.append(
/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.__events = result_left_part.append(
/home/vladimir/Workspace/retentioneering/retentioneering-tools-new-arch/src/eventstream/eventstream.py:216: FutureWarning: The frame.append method i

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,bc7bb3d8-2e78-4e10-b40e-87be85598fd8,start,0,start,2019-11-01 17:59:13.273932,219483890.0
2,8af8001f-6327-492c-94d2-8550fa2ff782,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,9491057d-392d-4d2c-ae44-cb6e4a858813,group_alias,3,add_to_cart,2019-11-01 17:59:29.502214,219483890.0
5,3ca3b4f6-8f41-406e-9d9f-102cbf9dfeef,raw,5,catalog,2019-11-01 17:59:32.557029,219483890.0
6,989a2a8d-7421-4fd2-9c0c-a60b3df63cce,start,6,start,2019-11-01 21:38:19.283663,964964743.0
...,...,...,...,...,...,...
45719,9eb195a5-ef6a-43e7-b346-8a54a68f904d,raw,45719,catalog,2020-04-29 12:47:40.975732,501098384.0
45720,68ceb511-78f5-491a-8bfa-e5710ed13d94,raw,45720,catalog,2020-04-29 12:48:01.809577,501098384.0
45721,92212279-af10-4b3b-be2c-f87491e699e1,raw,45721,main,2020-04-29 12:48:01.938488,501098384.0
45722,62766dd1-e4c7-4602-86cc-e76a9c009d5e,raw,45722,catalog,2020-04-29 12:48:06.595390,501098384.0
